<a href="https://colab.research.google.com/github/FabianaFerrente/ibis_pipeline/blob/master/deconv_ibis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
deconv = torchmfbd.Deconvolution('deconv.yaml')

# Patchify and add the frames
frames_patches = [None] * 2
for i in range(2):
    frames_patches[i] = patchify.patchify(frames[:, i, :, :, :], patch_size=64, stride_size=50, flatten_sequences=True)
    decSI.add_frames(frames_patches[i], id_object=i, id_diversity=0, diversity=0.0)


deconv.deconvolve(infer_object=False,
                 optimizer='adam',
                 simultaneous_sequences=16,
                 n_iterations=20)

obj = []
frames_back = []
for i in range(2):
    obj.append(patchify.unpatchify(deconv.obj[i], apodization=6).cpu().numpy())
    frames_back.append(patchify.unpatchify(frames_patches[i], apodization=0).cpu().numpy())

npix = obj[0][0, :, :].shape[0]

fig, ax = pl.subplots(nrows=2, ncols=2, figsize=(10, 10))
for i in range(2):
    ax[0, i].imshow(frames[0, i, 0, 0:npix, 0:npix])
    ax[1, i].imshow(obj[i][0, :, :])

deconv.write('output.fits')